In [1]:
##### DNN module

from keras.utils import to_categorical
from keras import backend as K
from keras.models import Model
from keras.applications import ResNet50
from keras.optimizers import Adagrad
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
##### Common moldule

import numpy as np
import os
import glob
import matplotlib.pyplot as plt

In [3]:
##### Path

train_db = 'database/combined_train_mx_240.npy'
label_db = 'database/combined_label_mx_240.npy'

flower_li = list(map(os.path.basename, glob.glob('database/image_data/train/*')))
flower_mapping = {flower_li[i]: i for i in range(5)}

print(flower_mapping)

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [4]:
##### Load data

flower_data   = np.load(train_db).astype('uint8')
flower_target = np.load(label_db).astype('uint8')

x_train, x_test, y_train, y_test = train_test_split(flower_data, flower_target, test_size=0.2, random_state=9527)

In [5]:
##### Dimenssion check

(np.shape(x_train), np.shape(x_test), np.shape(y_train), np.shape(y_test))

((2258, 240, 240, 3), (565, 240, 240, 3), (2258,), (565,))

In [6]:
##### Normalize / Categorize

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 5)
y_test = to_categorical(y_test, 5)

In [7]:
##### Parameter

num_class = 5
batch_size = 32
num_epoches = 10

In [8]:
datagen = ImageDataGenerator(
          rotation_range=40,
          width_shift_range=0.2,
          #height_shift_range=0.2,
          #rescale=1./255,
          #shear_range=0.2,
          #zoom_range=0.2,
          #horizontal_flip=True,
          #fill_mode='nearest'
          )

In [9]:
datagen.fit(x_train)

In [10]:
gen = datagen.flow((x_train, y_train), batch_size=16)

In [11]:
##### Model

model = Sequential()

'''
model.add(Conv2D(32, (5, 5), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
'''


model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_class))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 240, 240, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 240, 240, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 238, 238, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 238, 238, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 119, 119, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 119, 119, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 119, 119, 64)     

In [12]:
##### Compile
model.compile(loss='categorical_crossentropy',
              optimizer=Adagrad(),
              metrics=['accuracy'])

'''

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=num_epoches,
                    verbose=1,
                    validation_data=(x_test, y_test))
'''
model.fit_generator(gen,
          steps_per_epoch=int(len(x_train)/batch_size),
          epochs=num_epoches,
          verbose=1,
          validation_data=(x_test, y_test))



Epoch 1/10
70/70 [==============================] - 15s 212ms/step - loss: 30.2483 - accuracy: 0.2152 - val_loss: 1.6055 - val_accuracy: 0.2496
Epoch 2/10
70/70 [==============================] - 11s 164ms/step - loss: 1.6005 - accuracy: 0.2509 - val_loss: 1.6050 - val_accuracy: 0.2496
Epoch 3/10
12/70 [====>.........................] - ETA: 8s - loss: 1.5831 - accuracy: 0.2760

KeyboardInterrupt: 

In [12]:
##### Result

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.8217520403651009
Test accuracy: 0.25132742524147034


In [13]:
#####
#####
#####